my api key: wFKolR3bfa5XUZaFntmdo5AXd7kL506y1klYRd3y


In [4]:
!pip install semanticscholar -q

import os
from dotenv import load_dotenv
from semanticscholar import SemanticScholar


def get_semantic_scholar_client():
    """
    Initializes Semantic Scholar API client.

    Priority:
    1. Environment variable (recommended)
    2. Manual API key (fallback)
    3. Limited-access mode

    Returns:
        SemanticScholar object
    """

    # Load .env variables (if available)
    load_dotenv()

    # OPTION 1: Environment variable
    api_key = os.getenv("wFKolR3bfa5XUZaFntmdo5AXu7kL506y1klYRd3y")

    # OPTION 2: MANUAL KEY (PUT YOUR KEY HERE IF NEEDED)
    # ⚠ REMOVE BEFORE SUBMISSION
    MANUAL_API_KEY = ""  # <-- Paste your API key here if needed

    if api_key:
        print("🔑 API key loaded from environment.")
        return SemanticScholar(api_key=api_key)

    elif MANUAL_API_KEY:
        print("🔑 API key loaded manually.")
        return SemanticScholar(api_key=MANUAL_API_KEY)

    else:
        print("⚠ No API key found.")
        print("⚠ Running in LIMITED API mode.")
        return SemanticScholar()

# ============================================================
# MODULE 0: SEMANTIC SCHOLAR API CONFIGURATION
# ============================================================
"""
This module handles:
✔ Secure API key loading
✔ Semantic Scholar client initialization
✔ Safe fallback to limited-access mode

All other modules import and reuse this client.
"""

'\nThis module handles:\n✔ Secure API key loading\n✔ Semantic Scholar client initialization\n✔ Safe fallback to limited-access mode\n\nAll other modules import and reuse this client.\n'

In [5]:
# MODULE 1: Topic Input & Paper Search (LIMITED TO 50 PAPERS)

!pip install semanticscholar python-dotenv requests tabulate pandas -q

import json
import os
import pandas as pd
from tabulate import tabulate
from semanticscholar import SemanticScholar
from dotenv import load_dotenv


# ------------------------------------------------------------
# 0. CREATE PROJECT FOLDER STRUCTURE
# ------------------------------------------------------------
def create_project_folders():
    folders = [
        "data",
        "data/search_results",
        "data/raw_pdfs",
        "data/metadata"
    ]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)
    print("📁 Project folders verified/created.")


# ------------------------------------------------------------
# 1. SETUP API KEY
# ------------------------------------------------------------
def setup_api_key():
    load_dotenv()
    API_KEY = os.getenv("wFKolR3bfa5XUZaFntmdo5AXd7kL506y1klYRd3y")

    if not API_KEY:
        print("⚠ No API key found. Running with LIMITED API rate.")
        return SemanticScholar()
    else:
        print("🔑 API Key loaded successfully.")
        return SemanticScholar(api_key=API_KEY)


# ------------------------------------------------------------
# 2. SEARCH PAPERS (STRICT LIMIT = 50)
# ------------------------------------------------------------
def search_papers(topic, year_filter=None, open_access_only=False):
    sch = setup_api_key()

    MAX_PAPERS = 50
    print(f"\n🔎 Searching for papers on: '{topic}'")
    print(f"📡 Fetching up to {MAX_PAPERS} papers from Semantic Scholar...")

    try:
        results = sch.search_paper(
            query=topic,
            limit=MAX_PAPERS,
            fields=[
                "paperId", "title", "abstract", "year", "authors",
                "citationCount", "openAccessPdf", "url", "venue"
            ]
        )

        papers = []

        for paper in results:
            if len(papers) >= MAX_PAPERS:
                break  # HARD STOP

            # Apply optional filters
            if year_filter and paper.year != year_filter:
                continue
            if open_access_only and not paper.openAccessPdf:
                continue

            papers.append({
                "paperId": paper.paperId,
                "title": paper.title,
                "authors": [a["name"] for a in paper.authors] if paper.authors else [],
                "year": paper.year,
                "abstract": (paper.abstract[:300] + "...") if paper.abstract else "No abstract available",
                "citationCount": paper.citationCount,
                "venue": paper.venue,
                "url": paper.url,
                "pdf_url": paper.openAccessPdf["url"] if paper.openAccessPdf else None,
                "has_pdf": bool(paper.openAccessPdf)
            })

        print("\n📊 Search Completed!")
        print(f"   ➤ Total papers collected: {len(papers)}")
        print(f"   ➤ Papers with PDF: {sum(p['has_pdf'] for p in papers)}")

        return {
            "topic": topic,
            "total_results": len(papers),
            "papers": papers
        }

    except Exception as e:
        print(f"❌ Error during search: {e}")
        return None


# ------------------------------------------------------------
# 3. SAVE RESULTS TO JSON + CSV
# ------------------------------------------------------------
def save_search_results(data):
    topic_clean = data["topic"].replace(" ", "_")
    json_path = f"data/search_results/{topic_clean}.json"
    csv_path = f"data/search_results/{topic_clean}.csv"

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"💾 JSON saved to: {json_path}")

    df = pd.DataFrame(data["papers"])
    df.to_csv(csv_path, index=False)
    print(f"📊 CSV saved to: {csv_path}")

    return json_path, csv_path


# ------------------------------------------------------------
# 4. DISPLAY RESULTS
# ------------------------------------------------------------
def display_search_results(data):
    papers = data["papers"]

    print("\n" + "="*90)
    print(f"SEARCH SUMMARY: {data['topic']}")
    print("="*90)

    print(f"\n📌 Total Papers: {len(papers)}")
    print(f"📌 Papers with PDF: {sum(p['has_pdf'] for p in papers)}")
    print(f"📌 Papers without PDF: {sum(not p['has_pdf'] for p in papers)}")

    table_data = []
    for p in papers[:10]:
        table_data.append([
            p["title"][:40] + ("..." if len(p["title"]) > 40 else ""),
            p["year"],
            p["citationCount"],
            "Yes" if p["has_pdf"] else "No"
        ])

    print("\n📝 TOP PAPERS TABLE:")
    print(tabulate(table_data, headers=["Title", "Year", "Citations", "PDF"], tablefmt="pretty"))


# ------------------------------------------------------------
# 5. MAIN EXECUTION
# ------------------------------------------------------------
def main_search():
    create_project_folders()

    print("\n" + "="*80)
    print(" MODULE 1: TOPIC INPUT & PAPER SEARCH")
    print("="*80)

    topic = input("\nEnter research topic: ").strip() or "machine learning"

    year_input = input("Filter by year? (Press Enter to skip): ").strip()
    year_filter = int(year_input) if year_input.isdigit() else None

    oa_input = input("Open access only? (y/n): ").lower().strip()
    open_access_only = oa_input == "y"

    results = search_papers(
        topic,
        year_filter=year_filter,
        open_access_only=open_access_only
    )

    if not results or results["total_results"] == 0:
        print("\n❌ No papers found.")
        return None

    json_path, csv_path = save_search_results(results)
    display_search_results(results)

    print("\n✅ Module 1 Completed Successfully!")
    print(f"➡ JSON saved at: {json_path}")
    print(f"➡ CSV saved at: {csv_path}")
    print("➡ Proceed to Module 2 for downloading PDFs.")

    return results


# RUN MODULE 1
main_search()


📁 Project folders verified/created.

 MODULE 1: TOPIC INPUT & PAPER SEARCH

Enter research topic: environment
Filter by year? (Press Enter to skip): 
Open access only? (y/n): y
⚠ No API key found. Running with LIMITED API rate.

🔎 Searching for papers on: 'environment'
📡 Fetching up to 50 papers from Semantic Scholar...

📊 Search Completed!
   ➤ Total papers collected: 50
   ➤ Papers with PDF: 50
💾 JSON saved to: data/search_results/environment.json
📊 CSV saved to: data/search_results/environment.csv

SEARCH SUMMARY: environment

📌 Total Papers: 50
📌 Papers with PDF: 50
📌 Papers without PDF: 0

📝 TOP PAPERS TABLE:
+---------------------------------------------+------+-----------+-----+
|                    Title                    | Year | Citations | PDF |
+---------------------------------------------+------+-----------+-----+
| ape 5.0: an environment for modern phylo... | 2018 |   6905    | Yes |
| R: A language and environment for statis... | 2014 |  341592   | Yes |
| Cytoscape: 

{'topic': 'environment',
 'total_results': 50,
 'papers': [{'paperId': '191067949003f2351044690f235648c79ef7621c',
   'title': 'ape 5.0: an environment for modern phylogenetics and evolutionary analyses in R',
   'authors': ['E. Paradis', 'K. Schliep'],
   'year': 2018,
   'abstract': 'Summary\nAfter more than fifteen years of existence, the R package ape has continuously grown its contents, and has been used by a growing community of users. The release of version 5.0 has marked a leap towards a modern software for evolutionary analyses. Efforts have been put to improve efficiency,...',
   'citationCount': 6905,
   'venue': 'Bioinform.',
   'url': 'https://www.semanticscholar.org/paper/191067949003f2351044690f235648c79ef7621c',
   'pdf_url': 'https://academic.oup.com/bioinformatics/article-pdf/35/3/526/27699799/bty633.pdf',
   'has_pdf': True},
  {'paperId': '659408b243cec55de8d0a3bc51b81173007aa89b',
   'title': 'R: A language and environment for statistical computing.',
   'authors':

In [6]:
# ============================================
# MODULE 2: Paper Selection & PDF Download
# (FIXED & ROBUST VERSION)
# ============================================

import json
import os
import requests

SEARCH_DIR = "data/search_results"
DOWNLOAD_DIR = "downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# ------------------------------------------------------------
# 1. LOAD SEARCH RESULTS (SAFE VERSION)
# ------------------------------------------------------------
def load_search_results():
    if not os.path.exists(SEARCH_DIR):
        raise FileNotFoundError(
            "Search results folder not found. Run Module 1 first."
        )

    files = [f for f in os.listdir(SEARCH_DIR) if f.endswith(".json")]

    if not files:
        raise FileNotFoundError(
            "No search result JSON found. Run Module 1 first."
        )

    # Pick latest file automatically
    files.sort(key=lambda f: os.path.getmtime(os.path.join(SEARCH_DIR, f)), reverse=True)
    filepath = os.path.join(SEARCH_DIR, files[0])

    print(f"📂 Loading search results from: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    return data

# ------------------------------------------------------------
# 2. FILTER PAPERS WITH PDFs
# ------------------------------------------------------------
def filter_papers_with_pdf(papers):
    pdf_papers = []
    for p in papers:
        if p.get("pdf_url"):
            pdf_papers.append(p)
    print(f"📄 Papers with PDF: {len(pdf_papers)}")
    return pdf_papers

# ------------------------------------------------------------
# 3. DOWNLOAD PDFS
# ------------------------------------------------------------
def download_pdfs(limit=3):
    data = load_search_results()
    papers = filter_papers_with_pdf(data["papers"])

    downloaded = 0

    for i, paper in enumerate(papers[:limit]):
        pdf_url = paper["pdf_url"]
        title = paper["title"][:50].replace(" ", "_")

        filename = f"{DOWNLOAD_DIR}/{i+1}_{title}.pdf"

        try:
            print(f"\n⬇ Downloading: {paper['title'][:60]}...")
            r = requests.get(pdf_url, timeout=30)

            if r.status_code == 200 and r.content[:4] == b"%PDF":
                with open(filename, "wb") as f:
                    f.write(r.content)
                downloaded += 1
                print(f"   ✅ Saved as {filename}")
            else:
                print("   ❌ Not a valid PDF")

        except Exception as e:
            print("   ❌ Download failed:", e)

    print(f"\n📊 Total PDFs downloaded: {downloaded}")

# ------------------------------------------------------------
# RUN MODULE 2
# ------------------------------------------------------------
download_pdfs(limit=3)


📂 Loading search results from: data/search_results/environment.json
📄 Papers with PDF: 27

⬇ Downloading: ape 5.0: an environment for modern phylogenetics and evoluti...
   ❌ Not a valid PDF

⬇ Downloading: Cytoscape: a software environment for integrated models of b...
   ✅ Saved as downloads/2_Cytoscape:_a_software_environment_for_integrated_m.pdf

⬇ Downloading: WebArena: A Realistic Web Environment for Building Autonomou...
   ✅ Saved as downloads/3_WebArena:_A_Realistic_Web_Environment_for_Building.pdf

📊 Total PDFs downloaded: 2


In [7]:
# ============================================================
# MILESTONE 2 (WEEK 3–4)
# MODULE 3 + MODULE 4
# PDF TEXT EXTRACTION, ANALYSIS & CROSS-PAPER COMPARISON
# ============================================================

"""
WHAT THIS CODE DOES (HIGH-LEVEL):

1. Loads research paper PDFs downloaded in Module 2
2. Extracts text from each PDF using robust strategies
3. Cleans noisy PDF text
4. Extracts academic sections:
   - Title
   - Abstract
   - Introduction
   - Methods
   - Results
   - Conclusion
5. Stores extracted content in structured JSON format
6. Extracts key findings from Results & Conclusion
7. Validates correctness and completeness
8. Compares findings across all papers
9. Generates summary + comparison reports

THIS DIRECTLY SATISFIES:
✔ Text extraction module
✔ Section-wise parsing
✔ Structured storage
✔ Key finding extraction
✔ Cross-paper comparison
✔ Validation of correctness
"""

# ------------------------------------------------------------
# INSTALL REQUIRED LIBRARIES
# ------------------------------------------------------------
!pip install pymupdf pymupdf4llm tqdm -q

import os
import re
import json
from pathlib import Path
from collections import Counter
from datetime import datetime
from tqdm import tqdm
import fitz                     # PyMuPDF
import pymupdf4llm              # Layout-aware extraction

# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------
DOWNLOAD_DIR = "downloads"          # PDFs from Module 2
OUTPUT_DIR = "data/extracted"       # Structured output
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📁 Using PDF directory:", DOWNLOAD_DIR)
print("📁 Extracted data stored in:", OUTPUT_DIR)

# ------------------------------------------------------------
# 1. BASIC TEXT CLEANING (ENHANCEMENT)
# ------------------------------------------------------------
def clean_text(text):
    """
    Cleans raw PDF text:
    - Removes excessive spaces
    - Fixes broken hyphenated words
    - Removes unreadable characters
    """
    if not text:
        return ""

    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'-\s+', '', text)   # Fix line-break hyphens
    text = ''.join(c for c in text if ord(c) >= 32)
    return text.strip()

# ------------------------------------------------------------
# 2. PDF TEXT EXTRACTION
# ------------------------------------------------------------
def extract_text_from_pdf(pdf_path):
    """
    Extract text using TWO strategies and select best output:
    1) pymupdf4llm (layout-aware, better for research papers)
    2) Standard PyMuPDF extraction (fallback)
    """

    print(f"\n📄 Extracting text from: {pdf_path.name}")

    try:
        doc = fitz.open(pdf_path)
        extracted_versions = []

        # Strategy 1: Layout-aware extraction
        try:
            md_text = pymupdf4llm.to_markdown(str(pdf_path))
            if md_text and len(md_text) > 1000:
                extracted_versions.append(md_text)
                print("   ✔ Used layout-aware extraction")
        except:
            pass

        # Strategy 2: Regular extraction (fallback)
        raw_text = ""
        for page in doc[:min(40, len(doc))]:
            raw_text += page.get_text()

        if len(raw_text) > 1000:
            extracted_versions.append(raw_text)
            print("   ✔ Used standard text extraction")

        doc.close()

        if not extracted_versions:
            print("   ❌ No usable text found")
            return None

        # Select longest version (most content)
        best_text = max(extracted_versions, key=len)
        return clean_text(best_text)

    except Exception as e:
        print(f"   ❌ Extraction failed: {e}")
        return None

# ------------------------------------------------------------
# 3. SECTION-WISE TEXT EXTRACTION
# ------------------------------------------------------------
def extract_sections(text):
    """
    Extracts standard research paper sections using regex patterns
    """

    sections = {
        "title": "",
        "abstract": "",
        "introduction": "",
        "methods": "",
        "results": "",
        "conclusion": ""
    }

    lines = text.split("\n")

    patterns = {
        "abstract": r"\babstract\b",
        "introduction": r"\bintroduction\b",
        "methods": r"\b(methods?|methodology|experiment)\b",
        "results": r"\b(results?|findings)\b",
        "conclusion": r"\b(conclusion|discussion)\b"
    }

    boundaries = {}

    for i, line in enumerate(lines):
        for section, pattern in patterns.items():
            if re.search(pattern, line.lower()):
                boundaries.setdefault(section, i)

    sorted_sections = sorted(boundaries.items(), key=lambda x: x[1])

    for idx, (section, start) in enumerate(sorted_sections):
        end = sorted_sections[idx + 1][1] if idx + 1 < len(sorted_sections) else len(lines)
        content = " ".join(lines[start:end]).strip()
        if len(content) > 200:
            sections[section] = content[:5000]

    # Title extraction (first meaningful line)
    for line in lines[:10]:
        if 20 < len(line) < 200:
            sections["title"] = line
            break

    return sections

# ------------------------------------------------------------
# 4. KEY FINDINGS EXTRACTION (MODULE 4)
# ------------------------------------------------------------
def extract_key_findings(sections, max_findings=5):
    """
    Extracts important conclusions from Results + Conclusion
    """

    keywords = [
        "we found", "results show", "our results",
        "significant", "improves", "outperforms",
        "demonstrates", "indicates", "we conclude"
    ]

    findings = []
    combined = sections.get("results", "") + " " + sections.get("conclusion", "")
    sentences = re.split(r'(?<=[.!?])\s+', combined)

    for sentence in sentences:
        if any(k in sentence.lower() for k in keywords):
            findings.append(sentence.strip())
        if len(findings) >= max_findings:
            break

    return findings

# ------------------------------------------------------------
# 5. VALIDATION OF CORRECTNESS
# ------------------------------------------------------------
def validate_extraction(paper):
    """
    Ensures extracted data is complete and meaningful
    """

    return {
        "has_abstract": len(paper["sections"].get("abstract", "")) > 200,
        "has_multiple_sections": sum(len(v) > 200 for v in paper["sections"].values()) >= 2,
        "has_key_findings": len(paper["key_findings"]) > 0,
        "sufficient_text": paper["stats"]["total_chars"] > 1000
    }

# ------------------------------------------------------------
# 6. PROCESS A SINGLE PDF (END-TO-END)
# ------------------------------------------------------------
def process_pdf(pdf_path):
    """
    Complete processing of ONE research paper
    """

    raw_text = extract_text_from_pdf(pdf_path)
    if not raw_text:
        return None

    sections = extract_sections(raw_text)
    key_findings = extract_key_findings(sections)

    paper = {
        "paper_id": pdf_path.stem,
        "filename": pdf_path.name,
        "sections": sections,
        "key_findings": key_findings,
        "stats": {
            "total_chars": len(raw_text),
            "sections_found": sum(len(v) > 200 for v in sections.values())
        }
    }

    paper["validation"] = validate_extraction(paper)
    return paper

# ------------------------------------------------------------
# 7. CROSS-PAPER COMPARISON
# ------------------------------------------------------------
def compare_papers(papers):
    """
    Finds common themes across all papers
    """

    all_findings = []
    for paper in papers:
        all_findings.extend(paper["key_findings"])

    words = []
    for finding in all_findings:
        words.extend(re.findall(r'\b[a-zA-Z]{4,}\b', finding.lower()))

    common_terms = Counter(words).most_common(10)

    return {
        "total_papers": len(papers),
        "total_findings": len(all_findings),
        "common_terms": common_terms
    }

# ------------------------------------------------------------
# 8. RUN COMPLETE PIPELINE
# ------------------------------------------------------------
def run_milestone_2():
    """
    Runs complete Milestone-2 pipeline
    """

    pdf_files = list(Path(DOWNLOAD_DIR).glob("*.pdf"))

    print("\n==============================")
    print("MILESTONE 2 PIPELINE STARTED")
    print("==============================")
    print(f"📄 PDFs found: {len(pdf_files)}")

    if not pdf_files:
        print("❌ No PDFs found. Run Module 2 first.")
        return

    results = []

    for pdf in tqdm(pdf_files):
        paper = process_pdf(pdf)
        if paper:
            results.append(paper)
            with open(f"{OUTPUT_DIR}/{paper['paper_id']}.json", "w", encoding="utf-8") as f:
                json.dump(paper, f, indent=2)

    comparison = compare_papers(results)

    with open(f"{OUTPUT_DIR}/comparison_report.json", "w", encoding="utf-8") as f:
        json.dump(comparison, f, indent=2)

    print("\n✅ MILESTONE 2 COMPLETED SUCCESSFULLY")
    print(f"📊 Papers processed: {len(results)}")
    print("📁 Output stored in:", OUTPUT_DIR)

    return results, comparison

# ------------------------------------------------------------
# RUN
# ------------------------------------------------------------
results, comparison = run_milestone_2()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 7.5 MB/s eta 0:00:00
Consider using the pymupdf_layout package for a greatly improved page layout analysis.
📁 Using PDF directory: downloads
📁 Extracted data stored in: data/extracted

MILESTONE 2 PIPELINE STARTED
📄 PDFs found: 2


  0%|          | 0/2 [00:00<?, ?it/s]


📄 Extracting text from: 2_Cytoscape:_a_software_environment_for_integrated_m.pdf


 50%|█████     | 1/2 [00:03<00:03,  3.19s/it]

   ✔ Used layout-aware extraction
   ✔ Used standard text extraction

📄 Extracting text from: 3_WebArena:_A_Realistic_Web_Environment_for_Building.pdf


100%|██████████| 2/2 [00:16<00:00,  8.12s/it]

   ✔ Used layout-aware extraction
   ✔ Used standard text extraction

✅ MILESTONE 2 COMPLETED SUCCESSFULLY
📊 Papers processed: 2
📁 Output stored in: data/extracted


In [8]:
# ============================================================
# MILESTONE 2 (WEEK 3–4)
# PDF TEXT EXTRACTION, SECTION ANALYSIS & CROSS-PAPER COMPARISON
# ============================================================



# ------------------------------------------------------------
# 0. INSTALL REQUIRED LIBRARIES
# ------------------------------------------------------------
# PyMuPDF: robust PDF reading
# pymupdf4llm: layout-aware extraction (works well for structured research PDFs)
# tqdm: progress bar for loops
!pip install pymupdf pymupdf4llm tqdm -q

# ------------------------------------------------------------
# 1. IMPORT MODULES
# ------------------------------------------------------------
import os
import re
import json
from pathlib import Path
from collections import Counter
from tqdm import tqdm
import fitz                # PyMuPDF
import pymupdf4llm         # Layout-aware text extraction

# ------------------------------------------------------------
# 2. CONFIGURATION
# ------------------------------------------------------------
# Directory where PDFs are stored (downloaded in Module 2)
DOWNLOAD_DIR = "downloads"

# Directory to save structured JSON outputs
OUTPUT_DIR = "data/extracted"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📁 Using PDF directory:", DOWNLOAD_DIR)
print("📁 Extracted data stored in:", OUTPUT_DIR)

# ------------------------------------------------------------
# 3. TEXT CLEANING FUNCTION
# ------------------------------------------------------------
def clean_text(text):
    """
    Cleans the raw PDF text.

    Steps:
    1. Remove multiple consecutive spaces.
    2. Fix line-break hyphenation (e.g., "ex-\nample" -> "example").
    3. Remove non-printable/unreadable characters.

    Returns:
        Cleaned string.
    """
    if not text:
        return ""

    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # Remove hyphenation at line breaks
    text = re.sub(r'-\s+', '', text)

    # Remove unreadable characters
    text = ''.join(c for c in text if ord(c) >= 32)

    return text.strip()

# ------------------------------------------------------------
# 4. PDF TEXT EXTRACTION FUNCTION
# ------------------------------------------------------------
def extract_text_from_pdf(pdf_path):
    """
    Extract text from PDF using TWO strategies:

    Strategy 1: Layout-aware extraction (pymupdf4llm) → better for academic PDFs.
    Strategy 2: Standard PyMuPDF extraction → fallback if layout-aware fails.

    Returns:
        Best cleaned text (string) or None if extraction fails.
    """
    print(f"\n📄 Extracting text from: {pdf_path.name}")

    try:
        # Open PDF with PyMuPDF
        doc = fitz.open(pdf_path)
        extracted_versions = []

        # --- Strategy 1: Layout-aware extraction ---
        try:
            layout_text = pymupdf4llm.to_markdown(str(pdf_path))
            if layout_text and len(layout_text) > 1000:  # Only accept meaningful content
                extracted_versions.append(layout_text)
                print("   ✔ Used layout-aware extraction")
        except Exception as e:
            print(f"   ⚠ Layout-aware extraction failed: {e}")

        # --- Strategy 2: Standard PyMuPDF extraction (fallback) ---
        raw_text = ""
        for page in doc[:min(40, len(doc))]:  # Limit to first 40 pages for speed
            raw_text += page.get_text()
        if len(raw_text) > 1000:
            extracted_versions.append(raw_text)
            print("   ✔ Used standard text extraction")

        doc.close()

        if not extracted_versions:
            print("   ❌ No usable text found")
            return None

        # Select the longest text (most complete)
        best_text = max(extracted_versions, key=len)
        return clean_text(best_text)

    except Exception as e:
        print(f"   ❌ Extraction failed: {e}")
        return None

# ------------------------------------------------------------
# 5. SECTION-WISE EXTRACTION FUNCTION
# ------------------------------------------------------------
def extract_sections(text):
    """
    Extract standard research paper sections using regex.

    Sections:
        - Title: first meaningful line
        - Abstract
        - Introduction
        - Methods/Methodology/Experiment
        - Results/Findings
        - Conclusion/Discussion

    Returns:
        Dictionary of sections with content.
    """
    sections = {
        "title": "",
        "abstract": "",
        "introduction": "",
        "methods": "",
        "results": "",
        "conclusion": ""
    }

    lines = text.split("\n")

    # Regex patterns for section headings
    patterns = {
        "abstract": r"\babstract\b",
        "introduction": r"\bintroduction\b",
        "methods": r"\b(methods?|methodology|experiment)\b",
        "results": r"\b(results?|findings)\b",
        "conclusion": r"\b(conclusion|discussion)\b"
    }

    boundaries = {}  # Stores line indices of section starts

    # Detect section start lines
    for i, line in enumerate(lines):
        for section, pattern in patterns.items():
            if re.search(pattern, line.lower()):
                boundaries.setdefault(section, i)

    # Sort sections by start line
    sorted_sections = sorted(boundaries.items(), key=lambda x: x[1])

    # Extract section content based on line boundaries
    for idx, (section, start) in enumerate(sorted_sections):
        end = sorted_sections[idx + 1][1] if idx + 1 < len(sorted_sections) else len(lines)
        content = " ".join(lines[start:end]).strip()
        if len(content) > 200:  # Only keep meaningful content
            sections[section] = content[:5000]  # Limit size for JSON

    # Title: first meaningful line in first 10 lines
    for line in lines[:10]:
        if 20 < len(line) < 200:
            sections["title"] = line.strip()
            break

    return sections

# ------------------------------------------------------------
# 6. KEY FINDINGS EXTRACTION FUNCTION
# ------------------------------------------------------------
def extract_key_findings(sections, max_findings=5):
    """
    Extracts important results/conclusions from Results + Conclusion sections.

    Looks for sentences containing keywords like:
        - we found, results show, significant, improves, outperforms, indicates

    Returns:
        List of key sentences (max `max_findings`)
    """
    keywords = [
        "we found", "results show", "our results",
        "significant", "improves", "outperforms",
        "demonstrates", "indicates", "we conclude"
    ]

    findings = []
    combined_text = sections.get("results", "") + " " + sections.get("conclusion", "")
    sentences = re.split(r'(?<=[.!?])\s+', combined_text)

    for sentence in sentences:
        if any(k in sentence.lower() for k in keywords):
            findings.append(sentence.strip())
        if len(findings) >= max_findings:
            break

    return findings

# ------------------------------------------------------------
# 7. VALIDATION FUNCTION
# ------------------------------------------------------------
def validate_extraction(paper):
    """
    Checks whether extracted data is complete and meaningful.

    Criteria:
        - Abstract exists (>200 chars)
        - At least 2 sections exist
        - Key findings exist
        - Total text is sufficiently long (>1000 chars)

    Returns:
        Dictionary of validation flags
    """
    return {
        "has_abstract": len(paper["sections"].get("abstract", "")) > 200,
        "has_multiple_sections": sum(len(v) > 200 for v in paper["sections"].values()) >= 2,
        "has_key_findings": len(paper["key_findings"]) > 0,
        "sufficient_text": paper["stats"]["total_chars"] > 1000
    }

# ------------------------------------------------------------
# 8. PROCESS SINGLE PDF
# ------------------------------------------------------------
def process_pdf(pdf_path):
    """
    End-to-end processing of one research paper:
        1. Extract text
        2. Extract sections
        3. Extract key findings
        4. Collect stats
        5. Validate
    """
    raw_text = extract_text_from_pdf(pdf_path)
    if not raw_text:
        return None

    sections = extract_sections(raw_text)
    key_findings = extract_key_findings(sections)

    paper = {
        "paper_id": pdf_path.stem,
        "filename": pdf_path.name,
        "sections": sections,
        "key_findings": key_findings,
        "stats": {
            "total_chars": len(raw_text),
            "sections_found": sum(len(v) > 200 for v in sections.values())
        }
    }

    paper["validation"] = validate_extraction(paper)
    return paper

# ------------------------------------------------------------
# 9. CROSS-PAPER COMPARISON
# ------------------------------------------------------------
def compare_papers(papers):
    """
    Finds common themes across multiple papers:
        - Combines all key findings
        - Extracts most frequent words (>3 letters)

    Returns:
        Summary dictionary with total papers, total findings, and top 10 common terms
    """
    all_findings = []
    for paper in papers:
        all_findings.extend(paper["key_findings"])

    words = []
    for finding in all_findings:
        words.extend(re.findall(r'\b[a-zA-Z]{4,}\b', finding.lower()))

    common_terms = Counter(words).most_common(10)

    return {
        "total_papers": len(papers),
        "total_findings": len(all_findings),
        "common_terms": common_terms
    }

# ------------------------------------------------------------
# 10. RUN COMPLETE PIPELINE
# ------------------------------------------------------------
def run_milestone_2():
    """
    Runs the complete Milestone 2 pipeline:
        1. Process all PDFs
        2. Save JSONs for each paper
        3. Generate comparison report
    """
    pdf_files = list(Path(DOWNLOAD_DIR).glob("*.pdf"))

    print("\n==============================")
    print("MILESTONE 2 PIPELINE STARTED")
    print("==============================")
    print(f"📄 PDFs found: {len(pdf_files)}")

    if not pdf_files:
        print("❌ No PDFs found. Run Module 2 first.")
        return

    results = []

    for pdf in tqdm(pdf_files):
        paper = process_pdf(pdf)
        if paper:
            results.append(paper)
            # Save structured JSON for each paper
            with open(f"{OUTPUT_DIR}/{paper['paper_id']}.json", "w", encoding="utf-8") as f:
                json.dump(paper, f, indent=2)

    # Cross-paper comparison
    comparison = compare_papers(results)
    with open(f"{OUTPUT_DIR}/comparison_report.json", "w", encoding="utf-8") as f:
        json.dump(comparison, f, indent=2)

    print("\n✅ MILESTONE 2 COMPLETED SUCCESSFULLY")
    print(f"📊 Papers processed: {len(results)}")
    print("📁 Output stored in:", OUTPUT_DIR)

    return results, comparison

# ------------------------------------------------------------
# 11. EXECUTE PIPELINE
# ------------------------------------------------------------
results, comparison = run_milestone_2()


📁 Using PDF directory: downloads
📁 Extracted data stored in: data/extracted

MILESTONE 2 PIPELINE STARTED
📄 PDFs found: 2


  0%|          | 0/2 [00:00<?, ?it/s]


📄 Extracting text from: 2_Cytoscape:_a_software_environment_for_integrated_m.pdf


 50%|█████     | 1/2 [00:03<00:03,  3.16s/it]

   ✔ Used layout-aware extraction
   ✔ Used standard text extraction

📄 Extracting text from: 3_WebArena:_A_Realistic_Web_Environment_for_Building.pdf


100%|██████████| 2/2 [00:16<00:00,  8.38s/it]

   ✔ Used layout-aware extraction
   ✔ Used standard text extraction

✅ MILESTONE 2 COMPLETED SUCCESSFULLY
📊 Papers processed: 2
📁 Output stored in: data/extracted


In [12]:
"""
=========================================================
MILESTONE 3
AI-BASED SUMMARIZATION, ANALYSIS & LITERATURE REVIEW
=========================================================


"""

# --------------------------------------------------
# IMPORTS
# --------------------------------------------------
import os
import json
from pathlib import Path
from typing import Dict
from tqdm import tqdm

from google.colab import userdata
from openai import OpenAI

# --------------------------------------------------
# CONFIGURATION
# --------------------------------------------------
EXTRACTED_DIR = "data/extracted"
OUTPUT_DIR = "data/milestone_3"

SUMMARY_FILE = f"{OUTPUT_DIR}/paper_summaries.json"
ANALYSIS_FILE = f"{OUTPUT_DIR}/paper_analysis.json"
FINAL_REVIEW_FILE = f"{OUTPUT_DIR}/final_literature_review.txt"

MAX_SECTION_CHARS = 3500

# --------------------------------------------------
# INITIALIZATION (COLAB SECRETS SAFE)
# --------------------------------------------------
def initialize_environment():
    api_key = userdata.get("OPENAI_API_KEY")

    if not api_key:
        raise EnvironmentError(
            "OPENAI_API_KEY not found in Colab Secrets"
        )

    os.environ["OPENAI_API_KEY"] = api_key
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print("✅ OpenAI API key loaded securely from Colab Secrets")
    return OpenAI()

client = initialize_environment()

# --------------------------------------------------
# LLM CALL WRAPPER
# --------------------------------------------------
def call_llm(prompt: str, temperature: float = 0.3) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

# --------------------------------------------------
# SECTION-WISE SUMMARIZATION
# --------------------------------------------------
def summarize_section(section_name: str, text: str) -> str:
    prompt = f"""
    You are an academic research assistant.

    Summarize the {section_name} section of a research paper.
    Focus on:
    • Research problem
    • Methodology
    • Key results
    • Technical contribution

    Maintain formal academic tone.
    Limit to 120 words.

    TEXT:
    {text[:MAX_SECTION_CHARS]}
    """
    return call_llm(prompt, temperature=0.25)

# --------------------------------------------------
# PAPER-LEVEL SUMMARIZATION
# --------------------------------------------------
def summarize_paper(sections: Dict[str, str]) -> Dict[str, str]:
    summaries = {}

    for section, content in sections.items():
        if content and len(content.split()) > 80:
            summaries[section] = summarize_section(section, content)
        else:
            summaries[section] = "Section content insufficient for summarization."

    return summaries

# --------------------------------------------------
# CRITICAL ANALYSIS (KEY CONTRIBUTION MODULE)
# --------------------------------------------------
def analyze_paper(summary: Dict[str, str]) -> str:
    prompt = f"""
    Perform a critical analysis of the following paper summary.

    Identify:
    1. Key Contributions
    2. Strengths
    3. Limitations
    4. Research or practical implications

    Provide concise bullet points.

    SUMMARY:
    {json.dumps(summary, indent=2)}
    """
    return call_llm(prompt, temperature=0.35)

# --------------------------------------------------
# PROCESS ALL PAPERS
# --------------------------------------------------
def process_all_papers():
    extracted_files = list(Path(EXTRACTED_DIR).glob("*.json"))

    if not extracted_files:
        raise FileNotFoundError(
            "No extracted JSON files found from Milestone 2"
        )

    print("\n==============================")
    print(" MILESTONE 3 PIPELINE STARTED ")
    print("==============================")
    print(f"📄 Papers found: {len(extracted_files)}")

    all_summaries = {}
    all_analyses = {}

    for file in tqdm(extracted_files, desc="Processing papers"):
        with open(file, "r", encoding="utf-8") as f:
            paper_data = json.load(f)

        paper_id = paper_data.get("paper_id", file.stem)
        sections = paper_data.get("sections", {})

        # ---- Summarization ----
        paper_summary = summarize_paper(sections)
        all_summaries[paper_id] = paper_summary

        # ---- Analysis ----
        paper_analysis = analyze_paper(paper_summary)
        all_analyses[paper_id] = paper_analysis

    with open(SUMMARY_FILE, "w", encoding="utf-8") as f:
        json.dump(all_summaries, f, indent=2)

    with open(ANALYSIS_FILE, "w", encoding="utf-8") as f:
        json.dump(all_analyses, f, indent=2)

    print("✅ Paper-level summarization & analysis completed")
    return all_summaries, all_analyses

# --------------------------------------------------
# CROSS-PAPER SYNTHESIS (LITERATURE REVIEW)
# --------------------------------------------------
def generate_literature_review(
    summaries: Dict[str, Dict[str, str]],
    analyses: Dict[str, str]
) -> str:

    combined_text = ""

    for paper_id in summaries:
        combined_text += f"\nPAPER ID: {paper_id}\n"
        combined_text += json.dumps(summaries[paper_id], indent=2)
        combined_text += "\nANALYSIS:\n"
        combined_text += analyses[paper_id]
        combined_text += "\n\n"

    prompt = f"""
    You are an expert academic researcher.

    Write a comprehensive literature review based on the papers below.

    Your review must:
    • Identify major research trends
    • Compare methodologies
    • Highlight common limitations
    • Clearly state research gaps

    Use IEEE-style academic tone.
    Length: 800–1000 words.

    CONTENT:
    {combined_text[:12000]}
    """
    return call_llm(prompt, temperature=0.3)

# --------------------------------------------------
# MAIN PIPELINE
# --------------------------------------------------
def run_milestone_3():
    summaries, analyses = process_all_papers()

    print("\n==============================")
    print(" CROSS-PAPER SYNTHESIS STARTED ")
    print("==============================")

    final_review = generate_literature_review(summaries, analyses)

    with open(FINAL_REVIEW_FILE, "w", encoding="utf-8") as f:
        f.write(final_review)

    print("✅ Literature review generated successfully")
    print("📁 Output:", FINAL_REVIEW_FILE)

    print("\n==============================")
    print(" MILESTONE 3 COMPLETED ")
    print("==============================")

    return final_review

# --------------------------------------------------
# ENTRY POINT
# --------------------------------------------------
if __name__ == "__main__":
    run_milestone_3()


✅ OpenAI API key loaded securely from Colab Secrets

 MILESTONE 3 PIPELINE STARTED 
📄 Papers found: 3


Processing papers: 100%|██████████| 3/3 [00:56<00:00, 18.73s/it]


✅ Paper-level summarization & analysis completed

 CROSS-PAPER SYNTHESIS STARTED 
✅ Literature review generated successfully
📁 Output: data/milestone_3/final_literature_review.txt

 MILESTONE 3 COMPLETED 


In [18]:
"""
=========================================================
MILESTONE 4 (FINAL – STABLE VERSION)
Review, Refinement, Quality Evaluation & Final Report
=========================================================

"""

# --------------------------------------------------
# IMPORTS
# --------------------------------------------------
import os
import json
from typing import Dict
from google.colab import userdata
from openai import OpenAI

# --------------------------------------------------
# PATHS
# --------------------------------------------------
M3_DIR = "data/milestone_3"
M4_DIR = "data/milestone_4"

SUMMARY_FILE = f"{M3_DIR}/paper_summaries.json"
REVIEW_FILE = f"{M3_DIR}/final_literature_review.txt"

REVISED_FILE = f"{M4_DIR}/revised_sections.json"
QUALITY_FILE = f"{M4_DIR}/quality_report.json"
FINAL_REPORT_FILE = f"{M4_DIR}/final_report.txt"

os.makedirs(M4_DIR, exist_ok=True)

# --------------------------------------------------
# OPENAI INITIALIZATION (COLAB SAFE)
# --------------------------------------------------
def init_client():
    api_key = userdata.get("OPENAI_API_KEY")
    if not api_key:
        raise EnvironmentError("OPENAI_API_KEY missing in Colab Secrets")

    os.environ["OPENAI_API_KEY"] = api_key
    print("✅ OpenAI API key loaded securely from Colab Secrets")
    return OpenAI()

client = init_client()

# --------------------------------------------------
# LLM CALL
# --------------------------------------------------
def call_llm(prompt, temperature=0.3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

# --------------------------------------------------
# REVIEW & REVISION
# --------------------------------------------------
def critique_and_revise(section, text):
    prompt = f"""
    You are a senior academic reviewer.

    Critique the following {section} section and then provide
    an improved revised version.

    TEXT:
    {text}
    """
    return call_llm(prompt, temperature=0.35)

# --------------------------------------------------
# QUALITY EVALUATION
# --------------------------------------------------
def evaluate_quality(section, text):
    prompt = f"""
    Evaluate the quality of the following {section} section.

    Score (1–10):
    - Clarity
    - Technical Depth
    - Coherence
    - Academic Tone

    Provide short justification.

    TEXT:
    {text}
    """
    return call_llm(prompt, temperature=0.25)

# --------------------------------------------------
# LOAD MILESTONE 3 OUTPUTS
# --------------------------------------------------
def load_m3():
    with open(SUMMARY_FILE) as f:
        summaries = json.load(f)

    with open(REVIEW_FILE) as f:
        review_text = f.read()

    return summaries, review_text

# --------------------------------------------------
# MAIN REVIEW PIPELINE
# --------------------------------------------------
def run_milestone_4():
    summaries, review_text = load_m3()

    revised = {}
    quality = {}

    print("\n==============================")
    print(" MILESTONE 4 REVIEW STARTED ")
    print("==============================")

    for paper_id, sections in summaries.items():
        revised[paper_id] = {}
        quality[paper_id] = {}

        for section, text in sections.items():
            revised[paper_id][section] = critique_and_revise(section, text)
            quality[paper_id][section] = evaluate_quality(section, text)

    with open(REVISED_FILE, "w") as f:
        json.dump(revised, f, indent=2)

    with open(QUALITY_FILE, "w") as f:
        json.dump(quality, f, indent=2)

    final_report = generate_final_report(revised, review_text)

    with open(FINAL_REPORT_FILE, "w") as f:
        f.write(final_report)

    print("✅ MILESTONE 4 COMPLETED SUCCESSFULLY")
    print("📁 Output:", FINAL_REPORT_FILE)

    return final_report

# --------------------------------------------------
# FINAL REPORT
# --------------------------------------------------
def generate_final_report(revised, review_text):
    report = "FINAL AUTOMATED RESEARCH REVIEW REPORT\n"
    report += "=" * 50 + "\n\n"

    report += "LITERATURE REVIEW\n\n"
    report += review_text + "\n\n"

    report += "REVISED SECTIONS\n\n"

    for paper_id, sections in revised.items():
        report += f"\nPAPER ID: {paper_id}\n"
        report += "-" * 30 + "\n"
        for sec, content in sections.items():
            report += f"\n{sec.upper()}:\n{content}\n"

    return report

# --------------------------------------------------
# RUN
# --------------------------------------------------
if __name__ == "__main__":
    run_milestone_4()


✅ OpenAI API key loaded securely from Colab Secrets

 MILESTONE 4 REVIEW STARTED 
✅ MILESTONE 4 COMPLETED SUCCESSFULLY
📁 Output: data/milestone_4/final_report.txt
